# Spojování, agregace a řazení

## Pokročilé úpravy



In [1]:
import pandas as pd

url_food_nutrient = "https://kodim.cz/cms/assets/czechitas/python-data-1/python-pro-data-1/podmineny-vyber/podmineny-vyber/food_nutrient.csv"

food_nutrient = pd.read_csv(url_food_nutrient)
food_nutrient.head(3)

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1002)>

### Práce s chybějícími hodnotami



**`isna()`, `notna()`**

Obě metody vrací hodnoty `True` nebo `False`

In [3]:
food_nutrient_incomplete = food_nutrient[food_nutrient["amount"].isna()]
food_nutrient_incomplete.head(3)

NameError: name 'food_nutrient' is not defined

In [3]:
food_nutrient_complete = food_nutrient[food_nutrient["amount"].notna()]
food_nutrient_complete.head(3)

,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,loq,footnote,min_year_acquired,name,unit_name
4,1107528,1257,0.0,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN,"Fatty acids, total trans",G
5,1108007,1257,0.0,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN,"Fatty acids, total trans",G
6,1108158,1257,0.0,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN,"Fatty acids, total trans",G


**Jak chybějící hodnoty odstranit**

In [4]:
food_nutrient = food_nutrient.dropna(subset="amount")
food_nutrient.head(3)

,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,loq,footnote,min_year_acquired,name,unit_name
4,1107528,1257,0.0,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN,"Fatty acids, total trans",G
5,1108007,1257,0.0,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN,"Fatty acids, total trans",G
6,1108158,1257,0.0,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN,"Fatty acids, total trans",G


### Spojení dat

Nyní bychom chtěli všechny tři naše tabulky spojit do jedné. 

Nejprve si ukážeme, jak spojit tabulky pod sebe.

- Počet sloupců je ve výsledné tabulce stejný jako u spojovaných tabulek.
- Počet řádků odpovídá součtu řádků spojovaných tabulek.

V SQL používáme pro danou operaci klíčové slovo `UNION`, `pandas` používáme funkci `concat()`.

In [5]:
url_food_sample_100 = "https://kodim.cz/cms/assets/czechitas/python-data-1/python-pro-data-1/nacteni-dat/nacteni-dat/food_sample_100.csv"
url_food_other = "https://kodim.cz/cms/assets/czechitas/python-data-1/python-pro-data-1/spojovani/pokrocile-upravy/food_other.csv"

food_sample_100 = pd.read_csv(url_food_sample_100)
food_other = pd.read_csv(url_food_other)

Pozor na to, že v takto vzniklém DataFrame se nám rozbije index, protože se spojí za sebe indexy jednotlivých tabulek. 

Pokud chceme, aby pandas při spojování index přepočítal, musíme nastavit hodnotu parametru `ignore_index` na `True`.

In [6]:
food = pd.concat([food_sample_100, food_other], ignore_index=True)
food

,fdc_id,data_type,description,food_category_id,publication_date
0,2644829,sub_sample_food,"lentils, dry",16.0,2023-10-19
1,2347263,sub_sample_food,heavy cream,1.0,2022-10-28
2,2261954,sub_sample_food,"Flour, potato",11.0,2022-04-28
3,321470,sub_sample_food,"Salt, Iodized",2.0,2019-04-01
4,322951,sub_sample_food,Hot dogs beef,7.0,2019-04-01
...,...,...,...,...,...
9995,2661515,branded_food,"CHAI LATTE SUGAR FREE BLACK TEA CONCENTRATE, C...",NaN,2023-10-26
9996,2661730,branded_food,"APPLE BANANA ORGANIC FRUIT POUCHES, APPLE BANANA",NaN,2023-10-26
9997,2661782,branded_food,"SPINACH ORGANIC PUREE, SPINACH",NaN,2023-10-26
9998,2662107,branded_food,LIGHT MOZZARELLA LOW-MOISTURE PART-SKIM STRING...,NaN,2023-10-26


# Propojení dat

`pandas` umí `DataFrame` také propojit, což odpovídá SQL příkazu `JOIN`. 

- Tabulky můžeme spojit více způsoby. 
- Výsledná tabulka bude mít více sloupců.
- Počet řádků zavisí na typu propojení.


![Druhy propojeni](../../img/type_of_join_operations.png)

U operace `JOIN` jsou důležité dvě věci:

- Podle jakého sloupce (nebo jakých sloupců) dvě různé tabulky propojujeme.
- Co udělat v případě, že pro nějaké řádky nemám ve druhé tabulce odpovídající hodnotu.

In [7]:
print(food.shape)
print(food_nutrient.shape)

(10000, 5)
(135332, 13)


In [8]:
food_merged = pd.merge(food, food_nutrient, on="fdc_id", how="inner")
food_merged.shape

(135332, 17)

---
# Cvičení


## Značky a výrobci

Stáhni si data z tabulky `branded_food.csv`, která obsahuje data o konkrétních potravinách od výrobců. Tabulku načti do `pandas` tabulky `branded_food`.

Pro tabulku `branded_food` splň následující úkoly.

_Nápověda: Na body 2 a 3 využij metodu, kterou jsme si již ukazovali v předchozí lekci. Její název se skládá ze dvou slov - anglického výrazu pro hodnotu a pro počet._


In [9]:
url_branded_food = "https://kodim.cz/cms/assets/czechitas/python-data-1/python-pro-data-1/spojovani/excs/znacky/branded_food.csv"
branded_food = pd.read_csv(url_branded_food)



1. Zobraz si prvních několik řádků tabulky a podívej se na to, jaké jsou v ní sloupce a jaké jsou v nich hodnoty.


In [10]:
branded_food.head(2)

,fdc_id,brand_owner,brand_name,subbrand_name,gtin_upc,ingredients,not_a_significant_source_of,serving_size,serving_size_unit,household_serving_fulltext,branded_food_category,data_source,package_weight,modified_date,available_date,market_country,discontinued_date,preparation_state_code,trade_channel,short_description
0,1106032,BRAKEBUSH BROTHERS INC.,NaN,NaN,10038034720202,INGREDIENTS: BONELESS CHICKEN BREAST MEAT WITH...,NaN,85.0,g,NaN,Meat/Poultry/Other Animals Prepared/Processed,GDSN,NaN,2020-09-03,2020-11-13,United States,NaN,NaN,NaN,NaN
1,1107191,Mahco Inc,NaN,NaN,877448001674,"TOMATO PUREE (TOMATOES), CARROTS, ONIONS, SUN-...",NaN,125.0,g,NaN,Prepared Pasta & Pizza Sauces,LI,NaN,2020-09-22,2020-11-13,United States,NaN,NaN,NaN,NaN



2. Ve sloupci `brand_owner` jsou názvy výrobců potravin. Zjisti tři výrobce s největším počtem potravin v tabulce.


In [11]:
branded_food['brand_owner'].value_counts().head(3)

brand_owner
Wal-Mart Stores, Inc.    219
Target Stores            199
Meijer, Inc.             168
Name: count, dtype: int64



3. Ve sloupci `branded_food_category` jsou kategorie potravin. Zjisti pět kategorií s největších počtem potravin v tabulce.


In [12]:
branded_food['branded_food_category'].value_counts().head(5)

branded_food_category
Popcorn, Peanuts, Seeds & Related Snacks    504
Candy                                       415
Cheese                                      405
Ice Cream & Frozen Yogurt                   310
Cookies & Biscuits                          272
Name: count, dtype: int64

V tabulce je sloupec `fdc_id`, pomocí kterého ji můžeš propojit s tabulkou `food_merged`. 

Protože názvy jsou v obou tabulkách stejné, takže by bylo možné použít parametr `on`. Vyzkoušej si ale místo toho parametry `left_on` a `right_on`, kterým dáš stejnou hodnotu, tj. název sloupce `fdc_id`. 

Výsledek ulož do tabulky `food_merged_brands`.


In [13]:
food_merged_brands = pd.merge(food_merged, branded_food, right_on="fdc_id", left_on="fdc_id")


Pro tabulku `food_merged_brands` splň následující úkoly.

1. U výsledné tabulky `food_merged_brands` zkontroluj počet řádků a srovnej ho s původní tabulkou `food_merged`. Ubyly nějaké řádky? A čím to je?


In [14]:
print(food_merged_brands.shape)
print(food_merged.shape)

(129280, 36)
(135332, 17)


2. Nyní proveď operaci merge znovu, ale s parametrem `how` nastaveným na hodnotu `left`. Zkontroluj počet řádků a porovnej ho s počtem řádků tabulky `food_merged_brands`. Proč se počet liší?

In [15]:
food_merged_brands = pd.merge(food_merged, branded_food, right_on="fdc_id", left_on="fdc_id", how="left")
print(food_merged_brands.shape)
print(food_merged.shape)

(135332, 36)
(135332, 17)


## Přejmenování sloupce

Čím více tabulky propojujeme, tím více sloupců v nich máme. To může být časem až matoucí. 

Například máme v tabulce sloupec name, který označuje název výživné látky. 

Pro větší přehlednost ho přejmenujme na `nutrient_name`. K tomu slouží metoda `rename`. 

Níže je příklad toho, jak lze metodu použít. Pozorně si ho prohlédni. 

Metodu voláme pomocí tečkové notace, kterou už jsme využívali. 

```food_merged_brands = food_merged_brands.rename(columns={"stary_nazev": "novy_nazev"})```


Dále je potřeba zapsat název parametru (`columns`, protože chceme přejmenovat sloupce). Jako hodnotu vkládáme strukturu, která je označováno jako slovník. Jedná se o dvojici hodnot ve složených závorkách, která je oddělená dvojtečkou. Před dvojtečkou je starý název sloupce (v našem případě to bude name) a za dvojtečkou nový název sloupce (v našem případě to bude `nutrition_name`). Uprav tedy použití metody a spusť ho. Následně zkontroluj výsledek pomocí `.columns`.



In [16]:
food_merged_brands = food_merged_brands.rename(columns={"name": "nutrient_name"})

Jako bonus se zamysli nad tím, co by se mohlo stát, pokud na místo starého názvu zadáš sloupec, který v tabulce není (např. test). 

Jaké chování bys od metody očekával(a)? Jakmile to budeš mít rozmyšlené, použij metodu s neexistujícím názvem sloupce a zkontroluj, co se stalo.

# Odstranění sloupců

Pokud máme v tabulce zbytečné sloupce, můžeme je odstranit. 

Zbytečných sloupců se můžeme zbavit dvě způsoby:

1. Výběrem sloupců, které v tabulce chceme. To už jsme si ukazovali v předchozí lekci v části "Výběr sloupců".
2. Odebráním sloupců, které v tabulce nechceme. K tomu slouží metoda `


Níže je příklad použití metody `drop()`. Použijeme parametr `columns`, kterému zadáme seznam sloupců k odstranění.


```
food_merged_brands = food_merged_brands.drop(columns=["sloupec_1", "sloupec_2"])
```


Využij metodu k odstranění sloupců `footnote` a `min_year_acquired`. 


In [17]:
food_merged_brands = food_merged_brands.drop(columns=["footnote", "min_year_acquired"])



Pomocí vlastnosti `.columns` zkontroluj výsledek. 



In [18]:
food_merged_brands.columns

Index(['fdc_id', 'data_type', 'description', 'food_category_id',
       'publication_date', 'nutrient_id', 'amount', 'data_points',
       'derivation_id', 'min', 'max', 'median', 'loq', 'nutrient_name',
       'unit_name', 'brand_owner', 'brand_name', 'subbrand_name', 'gtin_upc',
       'ingredients', 'not_a_significant_source_of', 'serving_size',
       'serving_size_unit', 'household_serving_fulltext',
       'branded_food_category', 'data_source', 'package_weight',
       'modified_date', 'available_date', 'market_country',
       'discontinued_date', 'preparation_state_code', 'trade_channel',
       'short_description'],
      dtype='object')

Nakonec se podobně jako v předchozím cvičení zamysli nad tím, co by se mohlo stát, kdybys do seznamu vložila neexistující název sloupce, například test. 

Poté metodu `drop()` s neexistujícím názvem sloupce vyzkoušej. Překvapil tě výsledek?

## Agregace a řazení



### Agregace
Agregace je operace, která "sloučí" více řádků tabulky do jednoho. 

U agregace většinou slučujeme řádky podle nějakého konkrétního sloupce. 

In [19]:
food_merged_brands_protein = food_merged_brands[
    food_merged_brands["nutrient_name"] == "Protein"
]
food_merged_brands_protein.head(3)

,fdc_id,data_type,description,food_category_id,publication_date,nutrient_id,amount,data_points,derivation_id,min,...,branded_food_category,data_source,package_weight,modified_date,available_date,market_country,discontinued_date,preparation_state_code,trade_channel,short_description
15,335770,agricultural_acquisition,"Beans, Dry, Small Red, 471 (0% moisture)",16.0,2019-04-01,1003,20.8,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,335751,agricultural_acquisition,"Beans, Dry, Small Red, 646 (0% moisture)",16.0,2019-04-01,1003,22.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,335616,agricultural_acquisition,"Beans, Dry, Pinto, 678 (0% moisture)",16.0,2019-04-01,1003,22.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
food_merged_brands_protein_agg = food_merged_brands_protein.groupby(
    "branded_food_category"
)["amount"].mean()

food_merged_brands_protein_agg

branded_food_category
Alcohol                          0.684000
All Noodles                     11.107368
Baby/Infant  Foods/Beverages     1.140000
Bacon                           45.200000
Bacon, Sausages & Ribs          27.239630
                                  ...    
Weight Control                   3.080000
Wholesome Snacks                 1.485645
Yogurt                           5.596461
Yogurt (Perishable)              4.320000
Yogurt/Yogurt Substitutes        5.578333
Name: amount, Length: 193, dtype: float64

### Řazení

In [21]:
food_merged_brands_protein_agg.sort_values(ascending=False).head(10)

branded_food_category
Bacon                                                  45.200000
Green Supplements                                      44.590000
Drinks Flavoured - Ready to Drink                      32.900000
Energy, Protein & Muscle Recovery Drinks               31.831481
Bacon, Sausages & Ribs                                 27.239630
Canned Tuna                                            21.648889
Meat/Poultry/Other Animals  Prepared/Processed         21.522368
Nut & Seed Butters                                     20.651270
Cheese                                                 19.969457
Meat/Poultry/Other Animals – Unprepared/Unprocessed    19.640000
Name: amount, dtype: float64

In [22]:
food_merged_brands_protein.sort_values("amount", ascending=False).head()

,fdc_id,data_type,description,food_category_id,publication_date,nutrient_id,amount,data_points,derivation_id,min,...,branded_food_category,data_source,package_weight,modified_date,available_date,market_country,discontinued_date,preparation_state_code,trade_channel,short_description
65920,1593315,branded_food,"CHOCOLATE SHAKE, CHOCOLATE",NaN,2021-03-19,1003,87.50,NaN,70.0,NaN,...,"Energy, Protein & Muscle Recovery Drinks",LI,NaN,2018-06-18,2021-03-19,United States,NaN,NaN,NaN,NaN
83126,1842352,branded_food,"PEDIATRIC SHAKE BALANCED NUTRITION, VANILLA",NaN,2021-06-25,1003,87.50,NaN,70.0,NaN,...,Powdered Drinks,LI,237 mL,2021-05-05,2021-06-25,United States,NaN,NaN,NaN,NaN
116139,2399314,branded_food,ORGANIC LONG GRAIN WHITE RICE,NaN,2022-12-22,1003,86.67,NaN,70.0,NaN,...,Rice,LI,25 lbs/11.34 kg,2020-07-20,2022-12-22,United States,NaN,NaN,NaN,NaN
109094,2280607,branded_food,BIRTHDAY CAKE FLAVORED GOLD STANDARD 100% ISOL...,NaN,2022-05-23,1003,83.33,NaN,70.0,NaN,...,"Energy, Protein & Muscle Recovery Drinks",LI,1.58 LB/720 G,2022-02-03,2022-05-23,United States,NaN,NaN,NaN,NaN
86095,1918665,branded_food,"OMEGA NUTRITION, KEFIR CULTURED MILK STARTER",NaN,2021-07-29,1003,80.00,NaN,73.0,NaN,...,Milk Additives,LI,1.1 oz/30 g,2018-04-27,2021-07-29,United States,NaN,NaN,NaN,NaN


---

# Cvičení

## Lipidy a tuky

Podívej se nyní na to, které kategorie potravin obsahují nejvíce lipidů (tuků). 

Nejprve pomocí dotazu vytvořit novou tabulku `food_merged_brands_lipid`, do které pomocí dotazu vlož pouze řádky, které mají jako název výživné látky hodnotu `Total lipid (fat)`. 


In [23]:
food_merged_brands_lipid = food_merged_brands[
    food_merged_brands["nutrient_name"] == "Total lipid (fat)"
]
food_merged_brands_lipid.head(3)

,fdc_id,data_type,description,food_category_id,publication_date,nutrient_id,amount,data_points,derivation_id,min,...,branded_food_category,data_source,package_weight,modified_date,available_date,market_country,discontinued_date,preparation_state_code,trade_channel,short_description
14,335770,agricultural_acquisition,"Beans, Dry, Small Red, 471 (0% moisture)",16.0,2019-04-01,1004,1.26,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,2004789,sub_sample_food,"GRAPE JUICE, PURPLE, FROM CONCENTRATE WITH ADD...",9.0,2021-10-15,1004,0.25,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,335751,agricultural_acquisition,"Beans, Dry, Small Red, 646 (0% moisture)",16.0,2019-04-01,1004,1.27,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Poté proveď agregaci podle návu kategorie a seřaď výslednou tabulku tak, aby nahoře byly vidět kategorie s největším počtem tuků. 

In [24]:
food_merged_brands_lipid_agg = food_merged_brands_lipid.groupby(
    "branded_food_category"
)["amount"].mean()

food_merged_brands_lipid_agg.sort_values(ascending=False)

branded_food_category
Oils Edible                                        93.330000
Vegetable & Cooking Oils                           78.012391
Butter & Spread                                    66.552667
Butter/Butter Substitutes                          63.965000
Meat/Poultry/Other Animals – Prepared/Processed    59.520000
                                                     ...    
Green Supplements                                   0.000000
Health Care                                         0.000000
Herbs/Spices/Extracts                               0.000000
Honey                                               0.000000
Plant Based Water                                   0.000000
Name: amount, Length: 193, dtype: float64

Porovnej si výslednou tabulku s tabulkou `food_merged_brands_protein_agg`, kterou jsme vytvořili v rámci lekce. 

Podívej se, zda se některé kategorie objevují v obou tabulkách.

## Výrobci a kategorie

Nyní uvažuj, že si chceme udělat přehled o tom, jaký výrobce produkuje jaké typy potravin. 



Proveď agregaci tabulky `food_merged_brands` podle dvou sloupců: `brand_owner` a `branded_food_category`. 

Sloupce musíš metodě `food_merged_brands` zadat jako seznam, tj. musíš použít hranaté závorky. 




Dále vyber sloupec `fdc_id` pro provedení agregace a použij agregaci `nunique()`, který vrátí počet unikátních hodnot. 

Nakonec použij metodu `sort_values` s tím, že chceš data seřadit sestupně.

In [25]:
food_merged_brands.groupby(
    ["brand_owner", "branded_food_category"]
)["fdc_id"].nunique().sort_values(ascending=False)

brand_owner                branded_food_category               
Ferrara Candy Company      Candy                                   40
CAMPBELL SOUP COMPANY      Prepared Soups                          29
Danone US, LLC             Yogurt                                  24
Bimbo Bakeries USA, Inc.   Breads & Buns                           24
Wells Enterprises, Inc.    Ice Cream & Frozen Yogurt               24
                                                                   ..
Hempler Foods Group LLC    Sausages, Hotdogs & Brats                1
Heinz USA                  Pickles, Olives, Peppers & Relishes      1
Heinz Frozen Food Company  Canned Soup                              1
Heintz & Weber Co Inc      Ketchup, Mustard, BBQ & Cheese Sauce     1
pure batch LLC             Chocolate                                1
Name: fdc_id, Length: 5892, dtype: int64